# Jax tutorial

# jax.numpyを用いた自動微分

In [0]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random

In [0]:
!pip install numpyro
import numpyro

numpy.jaxとnumpyの違いは, 乱数を生成する部分.

乱数を生成するには, randomのメソッドに対してkey情報を渡すことが必要となる.

In [5]:
key = random.PRNGKey(0)
x = random.normal(key, (10, ))
print(x)

[-0.372111    0.2642311  -0.18252774 -0.7368198  -0.44030386 -0.15214427
 -0.6713536  -0.59086424  0.73168874  0.56730247]


3000*3000の行列の席を計算して, 速度を算出してみる.


In [6]:
size = 3000
x = random.normal(key, (size, size), dtype=np.float32)
%timeit np.dot(x, x.T).block_until_ready()

1 loop, best of 3: 636 ms per loop


jax.numpyの関数には, オリジナルのnumpy配列を渡すことも可能

In [7]:
import numpy as onp
x = onp.random.normal(size= (size, size)).astype(onp.float32)
%timeit np.dot(x, x.T).block_until_ready()

1 loop, best of 3: 734 ms per loop


上記の実行が遅い理由としては, データの転送をCPUからGPUに行っていることにより生じている.

NDArrayをdeviceのメモリ上に乗せたまま実行するには, device_putを指定してやれば良い.

In [8]:
from jax import device_put

x = onp.random.normal(size=(size, size)).astype(onp.float32)
x = device_put(x)
%timeit np.dot(x, x.T).block_until_ready()

1 loop, best of 3: 555 ms per loop


device_putの返り値はNDArrayかのように振る舞うが, あくまでCPU上の値をコピーしているだけである.
device_putはjit(lambda x: x)と同様の役割を果たすが, こちらの方が速度としては早くなる.


GPUがある場合は, 下記のように実行してやれば, CPU上での実行よりかは早くなる.

In [9]:
x = onp.random.normal(size=(size, size)).astype(onp.float32)
%timeit onp.dot(x, x.T)

1 loop, best of 3: 552 ms per loop


### jitデコレータについて

jaxはGPU上で透過的に動作することが可能.

関数に対して@jitデコレータを使うことで 
関数処理をコンパイルして,高速化することが出来る

In [10]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * np.where(x > 0, x, alpha * np.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

The slowest run took 15.72 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 7.35 ms per loop


jitのコンパイル処理は関数seluが一度呼び出された時に初めて実行され,
その後はキャッシュされる

In [11]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

The slowest run took 22.43 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.57 ms per loop


### gradを用いた自動微分演算

jaxではgradメソッドを使うことで, 自動微分の演算を行うことが出来る

In [12]:
def sum_logistic(x):
  return np.sum(1.0 / (1.0 + np.exp(-x)))

x_small = np.arange(3.)
derivative_fn = grad(sum_logistic)
print(x_small)
print(derivative_fn(x_small))

[0. 1. 2.]
[0.25       0.19661197 0.10499357]


値が正しいかどうかを実際に実装して検証してみることとする.

In [13]:
def first_finite_difference(f, x):
  eps = 1e-3
  return np.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps)
                    for v in np.eye(len(x))])

print(x_small)
print(first_finite_difference(sum_logistic, x_small))

[0. 1. 2.]
[0.24998187 0.1964569  0.10502338]


gradを用いることで, 簡単に自動微分の演算を行うことが出来る.

またgradはjitと自由に組み合わせて使うことが出来る.

In [14]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0))

-0.03532558


In [0]:
from jax import jacfwd, jacrev
def hessian(fun):
  return jit(jacfwd(jacrev(fun)))

### vmapを用いた自動ベクトル化

JAXのAPIには、もう1つ便利な変換がある.

これは配列の軸に沿って関数をマッピングするというおなじみのセマンティクスを持っているが, ループを外側に保つのではなく、ループを関数のプリミティブな操作の中に押し込むことで, パフォーマンスを向上させることが出来る.

jitを使って作成した場合、バッチ次元を手動で実装するのと同じくらいの速さになる.

In [0]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
  return np.dot(mat, v)

通常 `apply_matrix`のような関数があると, 
Pythonでバッチ次元をループさせることが出来るが, 

通常この実装は速度としては遅くなってしまう.

In [19]:
def natively_batched_apply_matrix(v_batched):
  return np.stack([apply_matrix(v) for v in v_batched])

print('Natively batched')
%timeit natively_batched_apply_matrix(batched_x).block_until_ready()

Natively batched
The slowest run took 19.63 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 3.97 ms per loop


`np.dot`と`@jit`デコレータを使うことで, バッチ次元は高速化できる

In [21]:
@jit
def batched_apply_matrix(v_batched):
  return np.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
The slowest run took 84.06 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 196 µs per loop


バッチに対応していない複雑な関数があったとしても, 
`vmap`を用いることで, 簡単にjitを用いた高速化が可能となる

In [22]:
@jit
def vmap_batched_apply_matrix(v_batched):
  return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
The slowest run took 51.30 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 265 µs per loop
